In [1]:
import pandas as pd 
from utils.load_db import LoadDb
from utils.load_infos import LoadInfos
from utils.utils import *

## Carregando dados do excel e instanciando a classe que conecta com o sqlite

In [2]:
data = LoadDb('database.db')
df = pd.read_excel('data.xlsx')
df

,Location,Longitude,Latitude,Arrived,Gallons of Gas in Tank at Arrival,Departed,Gallons of Gas in Tank at Departure
0,1,-75.948486,41.054502,NaN,NaN,7.16.2019 7:33,16.0
1,2,-80.837402,41.112469,7.16.2019 13:02,6.8,7.16.2019 13:48,16.0
2,3,-82.122803,41.393294,7.16.2019 15:10,13.7,7.16.2019 15:17,13.7
3,4,-85.034180,41.697526,7.16.2019 17:38,8.6,7.16.2019 19:09,8.6
4,5,-87.297363,41.574361,7.16.2019 21:40,4.1,7.17.2019 7:46,16.0
5,6,-89.187012,41.368564,7.17.2019 9:22,13.0,7.17.2019 9:46,13.0
6,7,-91.647949,41.672912,7.17.2019 12:28,8.2,7.17.2019 13:06,8.2
7,8,-93.614502,41.607228,7.17.2019 14:41,4.9,7.17.2019 14:46,16.0
8,9,-95.910645,41.483891,7.17.2019 16:57,12.2,7.17.2019 17:53,12.2
9,10,-96.405029,42.463993,7.17.2019 19:01,9.7,7.17.2019 19:14,9.7


## Inserindo os dados no database

In [3]:
data.create()

for i in df.values:
    data.insert(i[0], i[1], i[2], i[4], i[6], i[3], i[5])


## Carregando os dados em um dataframe

In [4]:
df = pd.read_sql('select * from locations', con=data.conn)
data.close()
df

,id_location,longitude,latitude,gallons_arrival,gallons_departure,arrived,departed
0,1,-75.948486,41.054502,NaN,16.0,None,7.16.2019 7:33
1,2,-80.837402,41.112469,6.8,16.0,7.16.2019 13:02,7.16.2019 13:48
2,3,-82.122803,41.393294,13.7,13.7,7.16.2019 15:10,7.16.2019 15:17
3,4,-85.03418,41.697526,8.6,8.6,7.16.2019 17:38,7.16.2019 19:09
4,5,-87.297363,41.574361,4.1,16.0,7.16.2019 21:40,7.17.2019 7:46
5,6,-89.187012,41.368564,13.0,13.0,7.17.2019 9:22,7.17.2019 9:46
6,7,-91.647949,41.672912,8.2,8.2,7.17.2019 12:28,7.17.2019 13:06
7,8,-93.614502,41.607228,4.9,16.0,7.17.2019 14:41,7.17.2019 14:46
8,9,-95.910645,41.483891,12.2,12.2,7.17.2019 16:57,7.17.2019 17:53
9,10,-96.405029,42.463993,9.7,9.7,7.17.2019 19:01,7.17.2019 19:14


## Calculando a distancia, tempo e galões gastos

In [5]:
cont = 1
aux = df.values
gallons = list()
miles =  list()
times = list()
arrived_departed = list()

for i in range(len(aux)-1):
    gallons.append(float(aux[i][4]) - float(aux[i+1][3]))
    miles.append(distance_miles(float(aux[i][1]), float(aux[i][2]), float(aux[i+1][1]), float(aux[i+1][2])))
    times.append(diff_time(aux[i+1][5], aux[i][6]))
    arrived_departed.append([i+1, i])
    
    # if cont == len(aux)-1:
    #     break

    cont += 1


## Carrengado os dados acima para o banco de dados

In [6]:
data = LoadInfos('database.db')
data.create()

In [7]:
for i in range(len(arrived_departed)):
    mh = miles[i] / times[i]
    data.insert(i, miles[i], round(times[i], 2), round(gallons[i], 2), mh, arrived_departed[i][0], arrived_departed[i][1])


## Selecionando o registro com a maior velocidade

In [8]:
df = pd.read_sql('SELECT distance, time, gallons, MAX(miles_hour) FROM infos', con=data.conn)
df[['longitude_arrived', 'latitude_arrived', 'longitude_departed', 'latitude_departed']] = data.search_arrival_depart()
df
data.close()

,distance,time,gallons,MAX(miles_hour),longitude_arrived,latitude_arrived,longitude_departed,latitude_departed
0,152.037015,2.35,5.1,64.696602,-82.122803,41.393294,-80.837402,41.112469


## Achando a quantidade de milhas por galão

In [9]:
miles_gallon = df['distance'] / df['gallons']
print(f'A quantidade de milhas por galão foi: {miles_gallon.loc[0]}')


A quantidade de milhas por galão foi: 29.811179435838593
